**ANALISIS REGRESI LOGISTIK BINER**

Karakteristik nilai pada variabel dependent nya bersifat biner

In [5]:
# 1. Persiapan Lingkungan dan Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import statsmodels.api as sm


Unduh Data "Network Intrusion Detection" pada website Kaggle :
https://www.kaggle.com/datasets/sampadab17/network-intrusion-detection

In [9]:
# Muat dataset
url = 'Train_data.csv'  # Ganti dengan URL dataset yang sesuai
data = pd.read_csv(url)

In [12]:
# Lihat 10 data pertama
print("10 Data Pertama:")
print(data.head(10))

10 Data Pertama:
   duration protocol_type     service flag  src_bytes  dst_bytes  land  \
0         0           tcp    ftp_data   SF        491          0     0   
1         0           udp       other   SF        146          0     0   
2         0           tcp     private   S0          0          0     0   
3         0           tcp        http   SF        232       8153     0   
4         0           tcp        http   SF        199        420     0   
5         0           tcp     private  REJ          0          0     0   
6         0           tcp     private   S0          0          0     0   
7         0           tcp     private   S0          0          0     0   
8         0           tcp  remote_job   S0          0          0     0   
9         0           tcp     private   S0          0          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                  25   
1               0       0    0  ...               

In [15]:
print(data.columns)

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'class'],
      dtype='object')


In [18]:
print(f"Jumlah fitur: {data.shape[1] - 1}")  # Mengurangi 1 jika ada kolom target


Jumlah fitur: 41


In [21]:
# 2. Pra-pemrosesan Data
# Menampilkan fitur kategorikal dan elemen uniknya
categorical_features = data.select_dtypes(include=['object']).columns

for feature in categorical_features:
    print(f"Fitur: {feature}")
    print(data[feature].unique())  # Menampilkan elemen unik dalam fitur
    print("-" * 50)  # Pembatas agar lebih rapi


Fitur: protocol_type
['tcp' 'udp' 'icmp']
--------------------------------------------------
Fitur: service
['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001']
--------------------------------------------------
Fitur: flag
['SF' 'S0' 'REJ' 'RSTR' 'SH' 'RSTO' 'S1' 'RSTOS0' 'S3' 'S2' 'OTH']
--------------------------------------------------
Fitur: class
['normal' 'anomaly']
--------------------------------------------------


In [25]:
class_mapping = {
    'normal': 0,
    'anomaly': 1
}

# Gunakan .replace()
data['class'] = data['class'].replace(class_mapping)

# Cek apakah mapping berhasil
print(data['class'].unique())

[0 1]


In [28]:
protocol_type_mapping = {
    'tcp': 0,
    'udp': 1,
    'icmp':2
}

# Gunakan .replace()
data['protocol_type'] = data['protocol_type'].replace(protocol_type_mapping)

# Cek apakah mapping berhasil
print(data['protocol_type'].unique())

[0 1 2]


In [31]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['flag'] = label_encoder.fit_transform(data['flag'])

# Cek hasil encoding
print(data['flag'].unique())

[ 9  5  1  4 10  2  6  3  8  7  0]


In [35]:
label_encoder = LabelEncoder()
data['service'] = label_encoder.fit_transform(data['service'])

# Cek hasil encoding
print(data['service'].unique())

[19 41 46 22 48 33 35 13 32 57 17 11 55 63  2 51  6 62 34 61  3 10 18  4
 29 14 20 64 56 23 15 65 25 26 12 27 30 54 31 28 52 59 21 16 40  9 39  5
  7 53  8 50 37 44 38  0 43 45 58 42 47 36 49  1 60 24]


karena elemen kategorik di dalam fitur flag dan service banyak, kita menggunakan sintaks enumerasi otomatis.
kalian juga bisa menerapkan enumerasi otomatis pada fitur dengan elemen kategorik yang sedikit.
pilih sesuai kenyamanan saja. 

In [40]:
# Lihat 10 data pertama
print("10 Data Pertama:")
print(data.head(10))

10 Data Pertama:
   duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0         0              0       19     9        491          0     0   
1         0              1       41     9        146          0     0   
2         0              0       46     5          0          0     0   
3         0              0       22     9        232       8153     0   
4         0              0       22     9        199        420     0   
5         0              0       46     1          0          0     0   
6         0              0       46     5          0          0     0   
7         0              0       46     5          0          0     0   
8         0              0       48     5          0          0     0   
9         0              0       46     5          0          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                  25   
1               0       0    0  ...                   1   
2 

In [43]:
# Periksa nilai yang hilang
print(data.isnull().sum())

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

Karena tidak ada data yang hilang, maka proses akan dilanjutkan.
Namun jika ada data yang hilang pada fitur tertentu, kita bisa mengisi dengan nilai rataan nya dengan sintaks sbb:

**Jika data yang hilang berada pada fitur 1 maka sintaksnya adalah:**

data['feature1'].fillna(data['feature1'].mean(), inplace=True)

In [48]:
# Karena variabel target adalah 'class', maka kita pisahkan dengan variabel fiturnya
X = data.drop(columns=['class'])  # Semua fitur kecuali target
y = data['class']  # Target (label)

# Cek bentuk data
print(X.shape, y.shape)

(25192, 41) (25192,)


In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cek ukuran data
print(X_train.shape, X_test.shape)

(20153, 41) (5039, 41)


In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Standarisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Jangan fit ulang, cukup transform

# Buat model dengan iterasi lebih banyak
model = LogisticRegression(max_iter=500)

# Latih model
model.fit(X_train_scaled, y_train)

# Prediksi data uji
y_pred = model.predict(X_test_scaled)

# Lihat koefisien regresi
print("Intercept:", model.intercept_)
print("Koefisien:", model.coef_)


Intercept: [0.21748384]
Koefisien: [[-0.12897204  1.1579947  -0.10260657  1.3542286   0.81198179  0.0488104
  -0.07411299  1.42336143  0.08384793  1.13458728  0.008934    0.83689791
   2.98144887 -0.0037957  -1.1891885  -2.28288992 -0.00635226 -0.04736772
  -0.12337084  0.          0.         -1.07044126  1.63780281 -0.57733604
  -0.77893167  3.21356573  0.36188336  1.93521275 -1.00764074 -0.27011962
   0.24235848  0.71442223 -1.54995449  0.94630204  0.12099564  0.80743355
   0.3579031   1.13923661  0.85291914  0.36243198  0.29223982]]


In [57]:
import pandas as pd

# Prediksi menggunakan model yang sudah dilatih
y_pred = model.predict(X_test_scaled)  # Gunakan X_test_scaled jika sudah distandarisasi

# Buat DataFrame untuk membandingkan hasil prediksi dengan nilai aktual
hasil_prediksi = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Tampilkan 10 baris pertama hasil prediksi
print(hasil_prediksi.head(10))


       Actual  Predicted
19064       1          1
11127       0          0
6517        1          1
2973        0          0
13339       1          1
19289       0          0
2166        0          0
5548        0          0
10887       0          0
2222        1          1


In [60]:
# 4. Evaluasi Model

from sklearn.metrics import accuracy_score

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy:.4f}")


Akurasi Model: 0.9536


Akurasi model dihitung dengan cara membagi Jumlah Prediksi Benar dengan Total Sampel Uji. Nilai mendekati 1 berarti model semakin akurat.